In [1]:
!pip install langchain langchain-core langchain-community
!pip install openai
!pip install pypdf
!pip install tiktoken
!pip install chromadb

In [3]:
from langchain.document_loaders import PyPDFLoader
import tiktoken
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import AzureOpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import AzureChatOpenAI
from langchain.chains import RetrievalQA

In [4]:
loader = PyPDFLoader('/content/[정책브리프 2021-04] 탄소중립 대응을 위한 정부 정책과 동향.pdf')
pages = loader.load_and_split()

tokenizer = tiktoken.encoding_for_model('gpt-4o-mini')

def tiktoken_len(text):
    tokens = tokenizer.encode(text)
    return len(tokens)

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50, length_function=tiktoken_len)
docs = text_splitter.split_documents(pages)

In [ ]:
import os

os.environ['OPENAI_API_KEY'] = ''
os.environ['OPENAI_API_VERSION'] = '2023-05-15'
os.environ['OPENAI_API_TYPE'] = 'azure'
os.environ['AZURE_OPENAI_ENDPOINT'] = 'https://labuser24-aiservice-002.openai.azure.com'

In [6]:
from langchain.embeddings import AzureOpenAIEmbeddings

embedding_model = AzureOpenAIEmbeddings(
    azure_deployment='dev-text-embedding-ada-002',
    chunk_size = 500,
)

<ipython-input-6-56a9d52e716c>:3: LangChainDeprecationWarning: The class `AzureOpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import AzureOpenAIEmbeddings``.
  embedding_model = AzureOpenAIEmbeddings(


In [7]:
db = Chroma.from_documents(docs, embedding_model, persist_directory='./db')

In [14]:
openai = AzureChatOpenAI(
    model_name = 'dev-gpt-4o-mini',
    deployment_name = 'dev-gpt-4o-mini',
    temperature = 0.2
)

In [15]:
qa_chain = RetrievalQA.from_chain_type(
    llm = openai,
    chain_type = 'stuff',
    retriever = db.as_retriever()
)

In [16]:
query = '탄소중립 사회란?'
result = qa_chain(query)
print(result['result'])

NotFoundError: Error code: 404 - {'error': {'code': 'DeploymentNotFound', 'message': 'The API deployment for this resource does not exist. If you created the deployment within the last 5 minutes, please wait a moment and try again.'}}